# ea

In [1]:
import os
import sys
import gc
import itertools
import pickle
import pathlib
import datetime
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv
load_dotenv()
sys.path.append(os.getenv('UTILS_PATH'))

import pandas as pd
import numpy as np
import cudf
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns

import line_notify

In [2]:
import builtins
import types

def imports():
    for name, val in globals().items():
        # module imports
        if isinstance(val, types.ModuleType):
            yield name, val

            # functions / callables
        if hasattr(val, '__call__'):
            yield name, val


def noglobal(f):
    '''
    ref: https://gist.github.com/raven38/4e4c3c7a179283c441f575d6e375510c
    '''
    return types.FunctionType(f.__code__,
                              dict(imports()),
                              f.__name__,
                              f.__defaults__,
                              f.__closure__
                              )

In [3]:
Ns = {}
Ns['cf_a'] = 12
Ns['ctf_a'] = 12
Ns['atfd_a'] = 12
Ns['atfp_a'] = 12
Ns['pa_a'] = 12

Ns['cf_w'] = 12
Ns['ctf_w'] = 12
Ns['atfd_w'] = 12
Ns['atfp_w'] = 12
Ns['pa_w'] = 12

Ns['cf_m'] = 12
Ns['ctf_m'] = 12
Ns['atfd_m'] = 12
Ns['atfp_m'] = 12
Ns['pa_m'] = 12

Ns['cf_y'] = 12
Ns['ctf_y'] = 12
Ns['atfd_y'] = 12
Ns['atfp_y'] = 12
Ns['pa_y'] = 12

ディレクトリ設定

In [4]:
INPUT_DIR = os.getenv('INPUT_DIR')
OUTPUT_DIR = os.getenv('OUTPUT_DIR')
#exp_name = os.path.dirname(__file__).split('/')[-1]

データ読み込み

In [5]:
articles = pd.read_csv(INPUT_DIR + 'articles.csv', dtype='object')
customers = pd.read_csv(INPUT_DIR + 'customers.csv')
transactions = pd.read_csv(INPUT_DIR + 'transactions_train.csv', dtype={'article_id':'str'}, parse_dates=['t_dat'])
sample = pd.read_csv(INPUT_DIR + 'sample_submission.csv')

In [6]:
first_week_sales_pred = pd.read_csv(OUTPUT_DIR + '1st_week_sales_pred_v004/result.csv', dtype={'article_id':'str'},  parse_dates=['1st_week_sales_dat'])

# 前処理

In [7]:
ALL_CUSTOMER = customers['customer_id'].unique().tolist()
ALL_ARTICLE = articles['article_id'].unique().tolist()

customer_ids = dict(list(enumerate(ALL_CUSTOMER)))
article_ids = dict(list(enumerate(ALL_ARTICLE)))

customer_map = {u: uidx for uidx, u in customer_ids.items()}
article_map = {i: iidx for iidx, i in article_ids.items()}

articles['article_id'] = articles['article_id'].map(article_map)
customers['customer_id'] = customers['customer_id'].map(customer_map)
transactions['article_id'] = transactions['article_id'].map(article_map)
transactions['customer_id'] = transactions['customer_id'].map(customer_map)
sample['customer_id'] = sample['customer_id'].map(customer_map)
first_week_sales_pred['article_id'] = first_week_sales_pred['article_id'].map(article_map) 

In [8]:
# 名寄せ
customers['fashion_news_frequency'] = customers['fashion_news_frequency'].str.replace('None','NONE')

In [9]:
customers['age10'] = str((customers['age'] // 10) * 10)
customers.loc[customers['age'].isnull(), 'age10'] = np.nan

In [10]:
# label_encoding
le_cols = ['product_type_name', 'product_group_name', 'graphical_appearance_name',
            'colour_group_name', 'perceived_colour_value_name', 'perceived_colour_master_name', 'department_name',
            'index_name', 'section_name', 'garment_group_name']
for c in le_cols:
    le = LabelEncoder()
    articles[c] = le.fit_transform(articles[c].fillna(''))


le_cols = ['club_member_status', 'fashion_news_frequency', 'postal_code', 'age10']
for c in le_cols:
    le = LabelEncoder()
    customers[c] = le.fit_transform(customers[c].fillna(''))

In [11]:
ALL_INDEX_GROUP_NAME = articles['index_group_name'].unique().tolist()
index_group_name_ids = dict(list(enumerate(ALL_INDEX_GROUP_NAME)))
index_group_name_map = {u: uidx for uidx, u in index_group_name_ids.items()}
articles['index_group_name'] = articles['index_group_name'].map(index_group_name_map)

In [12]:
customers['customer_type'] = customers['FN'].fillna(0).astype(int).astype(str) + \
                             customers['Active'].fillna(0).astype(int).astype(str) + \
                             customers['club_member_status'].fillna(0).astype(int).astype(str) + \
                             customers['fashion_news_frequency'].fillna(0).astype(int).astype(str) + \
                             customers['age10'].fillna(0).astype(int).astype(str)

le = LabelEncoder()
customers['customer_type'] = le.fit_transform(customers['customer_type'])

In [13]:
# transactionに紐づけ
transactions = transactions.merge(customers, on='customer_id', how='left')
transactions = transactions.merge(articles, on='article_id', how='left')

In [14]:
# text特徴量
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

detail_desc_n_dim = 15
text_col = 'detail_desc'
articles[text_col] = articles[text_col].str.lower()
tfidf_vec = TfidfVectorizer(ngram_range=(1,2), stop_words='english')
text_tfidf = tfidf_vec.fit_transform(articles[text_col].fillna('').values.tolist())
svd = TruncatedSVD(n_components=detail_desc_n_dim, algorithm='arpack',random_state=42)
text_svd = svd.fit_transform(text_tfidf)
text_svd_df = pd.DataFrame(text_svd, columns=[f'{text_col}_svd_{i}' for i in range(detail_desc_n_dim)])
text_svd_df = pd.concat([articles[['article_id']], text_svd_df], axis=1)

prod_name_n_dim = 15
text_col = 'prod_name'
articles[text_col] = articles[text_col].str.lower()
tfidf_vec = TfidfVectorizer(ngram_range=(1,2), stop_words='english')
text_tfidf = tfidf_vec.fit_transform(articles[text_col].fillna('').values.tolist())
svd = TruncatedSVD(n_components=prod_name_n_dim, algorithm='arpack',random_state=42)

text_svd = svd.fit_transform(text_tfidf)
text_svd_df_tmp = pd.DataFrame(text_svd, columns=[f'{text_col}_svd_{i}' for i in range(prod_name_n_dim)])
text_svd_df = pd.concat([text_svd_df, text_svd_df_tmp], axis=1)


In [19]:
# 学習データの作成
# 1週ずつローリングして学習データを生成
train_start = '2020-09-09'
valid_start = '2020-09-16'
valid_end = '2020-09-22'

# 正解データ作成
valid = transactions[(transactions['t_dat'] >= valid_start) & (transactions['t_dat'] <= valid_end)].copy()
valid = valid[['customer_id', 'article_id']].drop_duplicates()
#valid = valid.groupby('customer_id')['article_id'].apply(list).reset_index()
#valid = valid.sort_values('customer_id').reset_index(drop=True)

In [16]:
oof = pd.read_csv(OUTPUT_DIR + 'exp055/exp055_oof.csv')

,customer_id,article_id
31548013,330,76695
31548014,330,103575
31548015,330,100334
31548016,349,420
31548017,349,14240
...,...,...
31788318,1371091,93053
31788319,1371691,104961
31788321,1371721,104053
31788322,1371747,88521
